In [3]:
import os
import sys
import re
import nltk
import operator
import pickle as pkl
# import progressbar
import time
import numpy as np
import pandas as pd

In [3]:
def tokenizer(text):
    text = re.sub("[^a-zA-Z]+", " ", text)
    tokens = nltk.tokenize.word_tokenize(text)
    return tokens        

def preprocessing_txt(text):
    
    tokens = tokenizer(text)
    stemmer = nltk.stem.porter.PorterStemmer()
    stopwords = nltk.corpus.stopwords.words('english')
    new_text = ""
    for token in tokens:
        token = token.lower()
        if token not in stopwords:
#             print token
            new_text += stemmer.stem(token)
            new_text += " "
        
    return new_text

In [5]:
def inverted_index():
    """
    Creates a dictionary of words as key and name of the documents as items
    """
    inverted = {}
    docs_indexed = 0
    list_doc = os.listdir("C:\Users\Sricharan\Desktop\information_retrieval\Data\cranfieldDocs")
    total = len(list_doc)
    point = total / 100
    increment = total / 100
    indexer = {}
    for doc in list_doc:
#         sys.stdout.write('\r')
        doc_loc = "C:\Users\Sricharan\Desktop\information_retrieval\Data\cranfieldDocs" + str(doc)
        file_doc = open(doc_loc, "r")
        file_doc = preprocessing_txt(file_doc.read())
        tokens = tokenizer(file_doc)
        for word in tokens:
            if not inverted.__contains__(word):
                count = 1
                doclist = {}
                doclist[doc] = 1
                inverted[word] = doclist
            else:
                if doc in inverted[word]:
                    doclist = inverted[word]
                    doclist[doc] += 1
                    inverted[word] = doclist
                else:
                    count = 1
                    doclist = inverted[word]
                    doclist[doc] = count
                    inverted[word] = doclist
        docs_indexed += 1
        i = docs_indexed
        if(i % (point) == 0):
            sys.stdout.write("\r[" + "=" * (i / increment) + ">" +  " " * ((total - i)/ increment) + "]" +  str(100*i / float(len(list_doc))) + "%")
            sys.stdout.flush()
    return inverted

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (753482037.py, line 7)

In [ ]:
if __name__ == "__main__":
    with open("indexed_docs.p","wb") as handle:
        indexed_docs = inverted_index()
        pkl.dump(indexed_docs, handle)

In [ ]:
with open("indexed_docs.p","r") as dict_file:
    dictionary = pkl.load(dict_file)

In [7]:
def process_query(query_file):
    
    f = open(query_file,"r")
    query_ID = []
    query_result = []
    Time = []
    q_res = pd.DataFrame(columns = ["queryID","Time"])
    for j,query in enumerate(f):
        result = []
        q_ID = query.split()[0]
        query_ID.append(q_ID)
        query = preprocessing_txt(query)
        query = query.split()
        START = time.time()
        for i,word in enumerate(query):
            
            if i == 0:
                for i in dictionary[word].items():
                    result.append(i[0])
            else:
                temp = []
                for i in dictionary[word].items():
                    temp.append(i[0])
                result = list(set(result).intersection(set(temp)))
        END = time.time()
        f1 = open("output_inverted_index.txt",'a')
#         print str(q_ID) + "%d" % len(result)
        for res in result:
            f1.write(str(q_ID) + " " + str(res) + "\n")
        f1.close() 
        Time.append(float(END - START))
        query_result.append(result)
    q_res["queryID"] = query_ID
    q_res["Time"] = Time
    q_res.to_csv("inverted_index.csv",encoding='utf-8')
    result = pd.DataFrame(columns = ["query_ID","relevant_docs"])
    result["query_ID"] = query_ID
    result["relevant_docs"] = query_result
    return result

In [ ]:
if __name__ == "__main__":
    query_file = "query.txt"
    process_query(query_file)

In [9]:
def precision_and_recall(output_file,filename):
    """
    Args:
        output_file: file containing result for queries
    """
    
    output = open("output.txt",'r')
    query = open("query.txt","r")
    est_output = open(output_file)
    query_ID = []
    prerec = open(filename,"a")
    prerec.write("queryID" + " " + "precision" + " " + "recall" + "\n")
    for line in query:
        query_ID.append(line.split()[0])
    e_out = pd.DataFrame(columns = ["q_ID","Doc"])
    o_out = pd.DataFrame(columns=["q_ID","Doc"])
    query = []
    docs = []
    for line in est_output:
        query.append(line.split()[0])
        docs.append(line.split()[1])
    e_out['q_ID'] = query
    e_out["Doc"] = docs
    query = []
    docs = []
    for line in output:
        query.append(line.split()[0])
        docs.append(line.split()[1])
    o_out['q_ID'] = query
    o_out["Doc"] = docs
        
    for q_ID in query_ID:
        estimated = list(e_out[e_out['q_ID'] == q_ID]["Doc"])
#         print len(estimated)
        true = list(o_out[o_out["q_ID"] == q_ID]["Doc"])
#         print len(true)

        if len(estimated) == 0:
            precision = 0
        else:    
            precision = len(list(set(estimated).intersection(set(true))))/float(len(estimated))
        recall = len(list(set(estimated).intersection(set(true))))/float(len(true))
        prerec.write(str(q_ID) + " " + str(precision) + " " + str(recall) + "\n")
    prerec.close()
    output.close()
    est_output.close()

In [10]:
precision_and_recall("output_inverted_index.txt","inverted_index_precision_and_recall.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'output.txt'